# Import Library

In [2]:
import numpy as np
import pandas as pd
import json
import pickle
import jsonpickle
from io import StringIO
import matplotlib.pyplot as plt
from datamodel import OrderDepth, UserId, TradingState, Order, Trade, Observation
from typing import List
import string

# Import Data

In [5]:
def create_symbol_dfs(dataframe, symbols, col_name):
    # 创建一个空的列表来存储每个symbol的DataFrame
    dfs = []

    # 遍历给定的symbols列表
    for symbol in symbols:
        # 从原始DataFrame中筛选出对应于当前symbol的行
        filtered_df = dataframe[dataframe[col_name] == symbol]

        # 将筛选后的DataFrame添加到列表中
        dfs.append(filtered_df)

    # 返回包含每个symbol DataFrame的列表
    return dfs

In [6]:
trade_round1_03 = pd.read_csv('Data/round 5 data/trades_round_1_day_0_wn.csv', delimiter=';')
trade_round1_02 = pd.read_csv('Data/round 5 data/trades_round_1_day_-1_wn.csv', delimiter=';')
trade_round1_01 = pd.read_csv('Data/round 5 data/trades_round_1_day_-2_wn.csv', delimiter=';')

trade_round3_01 = pd.read_csv('Data/round 5 data/trades_round_3_day_0_wn.csv', delimiter=';')
trade_round3_02 = pd.read_csv('Data/round 5 data/trades_round_3_day_1_wn.csv', delimiter=';')
trade_round3_03 = pd.read_csv('Data/round 5 data/trades_round_3_day_2_wn.csv', delimiter=';')

trade_round4_01 = pd.read_csv('Data/round 5 data/trades_round_4_day_1_wn.csv', delimiter=';')
trade_round4_02 = pd.read_csv('Data/round 5 data/trades_round_4_day_2_wn.csv', delimiter=';')
trade_round4_03 = pd.read_csv('Data/round 5 data/trades_round_4_day_3_wn.csv', delimiter=';')

# Data Preprocessing

In [10]:
round_1 = pd.concat([trade_round1_01, trade_round1_02, trade_round1_03])
round_3 = pd.concat([trade_round3_01, trade_round3_02, trade_round3_03])
round_4 = pd.concat([trade_round4_01, trade_round4_02, trade_round4_03])

In [16]:
round_4.head()

,timestamp,buyer,seller,symbol,currency,price,quantity
0,800,Valentina,Valentina,COCONUT_COUPON,SEASHELLS,636.0,5
1,900,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,634.0,17
2,1100,Valentina,Ruby,COCONUT_COUPON,SEASHELLS,635.0,17
3,1900,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,635.0,16
4,2000,Valentina,Vinnie,COCONUT_COUPON,SEASHELLS,637.0,14


In [13]:
all_rounds = pd.concat([round_1, round_3, round_4])

unique_buyers = set(all_rounds['buyer'])
unique_sellers = set(all_rounds['seller'])

traders = unique_buyers.union(unique_sellers)

In [67]:
rounds = {
    1: round_1,
    3: round_3,
    4: round_4
}

# Get Trading Data

In [ ]:
def trading_history(df, trader_name, symbol):
    # Filter the DataFrame for trades involving the specified trader and symbol
    # Create a copy of the trades to avoid SettingWithCopyWarning
    trades = df[(df['symbol'] == symbol) & ((df['buyer'] == trader_name) | (df['seller'] == trader_name))].copy()
    
    # Initialize the trader's capital
    initial_capital = 0
    initial_position = 0
    capital_history = []
    position_history = []
    profit_history = []

    # Process each trade to update capital based on buying or selling
    for index, row in trades.iterrows():
        total_value = row['price'] * row['quantity']  # Calculate total transaction value
        quantity = row['quantity']
        
        if row['buyer'] == trader_name:
            # Trader is buying, capital decreases
            initial_capital -= total_value
            initial_position += quantity
        if row['seller'] == trader_name:
            # Trader is selling, capital increases
            initial_capital += total_value
            initial_position -= quantity
            
        capital_history.append(initial_capital)
        position_history.append(initial_position)
        profit = initial_capital + row['price'] * initial_position
        profit_history.append(profit)
    
    # Assign capital history to trades DataFrame
    trades['capital'] = capital_history
    trades['position'] = position_history
    trades['profit'] = profit_history
    
    # Select relevant columns to show the trade and capital progression
    result = trades[['timestamp', 'buyer', 'seller', 'price', 'quantity', 'capital', 'position', 'profit']].copy()
    return result

In [73]:
# Get Trader Name
round_number = 3
product_name = 'GIFT_BASKET'

temp = rounds[round_number].copy()
temp = temp[temp['symbol'] == product_name]

coco_buyers = set(temp['buyer'])
coco_sellers = set(temp['seller'])
coco_buyers.union(coco_sellers)

{'Rhianna', 'Ruby', 'Vinnie', 'Vladimir'}

In [77]:
history = trading_history(temp, 'Vladimir', product_name)